In [305]:
import subprocess
import os
import shutil
import os
import zipfile


def zip_folder(folder_path, zip_file_name):
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(zip_file_name, "w", zipfile.ZIP_DEFLATED) as zipf:
        # Walk through the directory
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create the complete filepath of file in directory
                file_path = os.path.join(root, file)
                # Add file to zip
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

In [306]:
task_name = "home_data_3.2.5"
type_of_task = "function"  # "function"

solution_text = r"""
def bin_ip(ips):
    ip_dict = {}
    for ip in ips:
        octets = [int(o) for o in ip.split(".")]
        ip_dict.update({("{:08b}" * 4).format(*octets): ip})

    return dict(sorted(ip_dict.items()))
"""


legend = r"""Иван работает сетевым администратором. 
От сервера ему приходит список IP-адресов в виде строк, которые ему нужно отсортировать в порядке возрастания. 
Однако обычная сортировка дала неверный результат, поэтому ему пришлось использовать конвертацию адреса в двоичный вид.

Повторите за Иваном и напишите функцию \verb|bin_ip|, которая принимает на вход список из IP-адресов в виде строк, а возвращает отсортированный словарь, в котором ключом будет строка с двоичным видом IP-адреса, а значением — сам IP-адрес. 
Бинарный вид IP-адреса получается из "склеивания" вместе бинарных значений каждого из четырех десятичных октетов."""

input_info = r"\verb|list[str]|"
output_info = r"\verb|dict[str, str]|"
additional_notes = r"Ваше решение должно содержать только функцию с именем \verb|bin_ip|, без её вызова или ввода/вывода данных с командной строки."

test_cases = [
    '["192.168.1.1", "192.168.0.101"]',
    '["10.1.1.1", "10.1.10.1", "10.1.2.1", "10.1.11.1"]',
    '["192.168.1.1", "10.0.0.1", "172.16.0.1", "255.255.255.255", "8.8.8.8"]',
    '["255.255.255.255", "172.16.0.1", "192.168.1.1", "10.0.0.1", "8.8.8.8"]',
    '["192.168.1.10", "10.0.0.10", "172.16.0.10", "8.8.8.10"]',
    '["0.0.0.0", "255.255.255.255", "127.0.0.1", "192.168.1.1", "10.0.0.1"]',
    '["192.168.0.1", "192.168.1.1", "192.168.2.1", "192.168.3.1"]',
    '["192.168.1.1", "192.168.1.1", "192.168.1.1", "192.168.1.1"]',
    '["10.0.0.1", "100.100.100.100", "200.200.200.200", "250.250.250.250"]',
]

In [307]:
# Копируем папку
dirname = task_name
src_dir = "./sample"
dst_dir = "./" + dirname
# try:
#     shutil.rmtree(dst_dir)
#     os.remove(f"./{task_name}.zip")
# except FileNotFoundError:
#     pass
shutil.copytree(src_dir, dst_dir)

'./home_data_3.2.5'

In [308]:
# Поменять ./new_task/solutions/main.py


with open(f"./{dirname}/solutions/main.py", mode="w", encoding="utf-8") as file:
    file.write(solution_text)

In [309]:
# Сгенерить тесты


# TODO: LLM-кой сделать тесты
def create_test_cases() -> list[str]:
    pass


for i in range(len(test_cases)):
    with open(f"./{dirname}/tests/{i + 1:02}", mode="w", encoding="utf-8") as file_test:
        file_test.write(test_cases[i])

In [310]:
# Сгенерить ответы


# К этому моменту в папке ответов еще нет, так что это количество именно тестов
n_tests = len([x for x in os.listdir(f"./{dirname}/tests/") if not x.endswith(".a")])


if type_of_task == "function":
    with open(f"./{dirname}/files/participantSolution.py", mode="w", encoding="utf-8") as file:
        file.write(solution_text)

    commands = [
        f"python ./{dirname}/files/run_tests.py < ./{dirname}/tests/{test_number:02} > ./{dirname}/tests/{test_number:02}.a"
        for test_number in range(1, n_tests + 1)
    ]

    for cmd in commands:
        process = subprocess.run(cmd, shell=True, text=True, capture_output=True)

    subprocess.run(f"del ./{dirname}/files/participantSolution.py", shell=True, text=True, capture_output=True)
else:
    # basic prog task
    commands = [
        f"python ./{dirname}/solutions/main.py < ./{dirname}/tests/{test_number:02} > ./{dirname}/tests/{test_number:02}.a"
        for test_number in range(1, n_tests + 1)
    ]
    for cmd in commands:
        process = subprocess.run(cmd, shell=True, text=True, capture_output=True)
        print(process)

In [311]:
# Копирование примера
import shutil

src_test = f"./{dirname}/tests/01"
src_ans = f"./{dirname}/tests/01.a"
dst_test = f"./{dirname}/statements/russian/example.01"
dst_ans = f"./{dirname}/statements/russian/example.01.a"
shutil.copy(src_test, dst_test)
shutil.copy(src_ans, dst_ans)

'./home_data_3.2.5/statements/russian/example.01.a'

In [312]:
# Изменяем problem.xml
with open(f"./{dirname}/problem.xml", mode="r", encoding="utf-8") as file:
    content = file.read()


content = content.replace("placeholder_task_name", task_name)
content = content.replace("placeholder_ntests", str(n_tests - 1))

task_price = 1
if task_price > 1:
    content = content.replace("checker_1.py", f"checker_{task_price}.py")
    content = content.replace("files/checker_1.py", f"checker_{task_price}.py")

with open(f"./{dirname}/problem.xml", mode="w", encoding="utf-8") as file:
    file.write(content)

In [313]:
# Поменять условие
with open(f"./{dirname}/statements/russian/problem.tex", mode="r", encoding="utf-8") as file:
    content = file.read()


content = content.replace("placeholder_legend", legend)
content = content.replace("placeholder_input", input_info)
content = content.replace("placeholder_output", output_info)
content = content.replace("placeholder_note", additional_notes)

with open(f"./{dirname}/statements/russian/problem.tex", mode="w", encoding="utf-8") as file:
    file.write(content)

In [314]:
zip_folder(f"{dst_dir}", f"{task_name}.zip")